## Set-Up

In [ ]:
from scripts import models, sql

sql_coder_dir = "../models/sqlcoder"
sql_model, sql_token = models.get_coder(sql_coder_dir)

sql_data_path = "../dataset/nba_players.sqlite"
db_schema_path = "../structure/player_data_structure.txt"

with open(db_schema_path, 'r') as file:
    db_schema = file.read()

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [ ]:
llm_path = "../models/Meta-Llama-3-8B-Instruct.gguf"
llm = models.get_llm(llm_path)

llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from Meta-Llama-3-8B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attention.

In [ ]:
sql_prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
{schema}

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

llm_prompt = """Q: ### Task
Answer this question:  "{question}" using the retrieved context as ground truth: \n
CONTEXT: \n{context}\n
Don't mention the context in your answer, just give a response based on the data provided.
A:
"""

def format_llm_prompt(prompt_structure, retrieved_context, question):
    return prompt_structure.format(context=retrieved_context, question=question)


def format_sql_prompt(prompt_structure, schema, question):
    return prompt_structure.format(schema=schema, question=question)


def answer(data_path, db_schema, sql_model, sql_token, sql_prompt, llm, llm_prompt, question, print_prompts=False):
    sql_prompt = format_sql_prompt(sql_prompt, db_schema, question)
    sql_query = models.get_sql_query(sql_model, sql_token, sql_prompt)

    retreived_context = sql.get_query_output(data_path, sql_query)
    if retreived_context is False or len(retreived_context) == 0:
        return "Unable to generate SQL query"

    llm_prompt = format_llm_prompt(llm_prompt, retreived_context.to_string(), question)

    if print_prompts:
      print(f"SQL Query:\n{sql_query}")
      print(f"\nOutput:\n{retreived_context}")
      print(f"LLM Prompt:\n{llm_prompt}")

    return models.get_llm_response(llm, llm_prompt)

## Examples

In [ ]:
q = """
Who is the player that scored the most points in a game during the 2019 season
and in which game (when and against who)?
"""

response = answer(sql_data_path,
                  db_schema,
                  sql_model,
                  sql_token,
                  sql_prompt,
                  llm,
                  llm_prompt,
                  q)

print(response)


llama_print_timings:        load time =   40312.73 ms
llama_print_timings:      sample time =     109.78 ms /    51 runs   (    2.15 ms per token,   464.58 tokens per second)
llama_print_timings: prompt eval time =   40312.58 ms /   104 tokens (  387.62 ms per token,     2.58 tokens per second)
llama_print_timings:        eval time =   33782.76 ms /    50 runs   (  675.66 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =   74811.67 ms /   154 tokens


The player who scored the most points in a game during the 2019 season is James Harden with 61 points. This occurred on March 22, 2019, when he played against the San Antonio Spurs (HOU vs. SAS).


In [ ]:
q = """
Who is the player that scored the most points during the 2017 season
and how many points did he scored overall?
"""

response = answer(sql_data_path,
                  db_schema,
                  sql_model,
                  sql_token,
                  sql_prompt,
                  llm,
                  llm_prompt,
                  q)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   40312.73 ms
llama_print_timings:      sample time =      61.41 ms /    29 runs   (    2.12 ms per token,   472.24 tokens per second)
llama_print_timings: prompt eval time =   26596.50 ms /    66 tokens (  402.98 ms per token,     2.48 tokens per second)
llama_print_timings:        eval time =   16447.38 ms /    28 runs   (  587.41 ms per token,     1.70 tokens per second)
llama_print_timings:       total time =   43422.52 ms /    94 tokens


The player who scored the most points during the 2017 season is James Harden. He scored a total of 2490 points overall.


In [61]:
q = """
Show me the top 5 players of the 2020 season in terms of points per game
"""

response = answer(sql_data_path,
                  db_schema,
                  sql_model,
                  sql_token,
                  sql_prompt,
                  llm,
                  llm_prompt,
                  q)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   40312.73 ms
llama_print_timings:      sample time =     220.79 ms /   106 runs   (    2.08 ms per token,   480.10 tokens per second)
llama_print_timings: prompt eval time =   43043.74 ms /   108 tokens (  398.55 ms per token,     2.51 tokens per second)
llama_print_timings:        eval time =   65267.91 ms /   105 runs   (  621.60 ms per token,     1.61 tokens per second)
llama_print_timings:       total time =  109738.80 ms /   213 tokens


The top 5 players of the 2020 season in terms of points per game are:

1. Bradley Beal - 33.187500
2. Damian Lillard - 31.578947
3. James Harden - 31.184211
4. Trae Young - 31.121212
5. Russell Westbrook - 29.193548

Note: The ranking is based on the average points per game for each player during the 2020 season, as provided in the data.


In [ ]:
q = """
Show me the top 5 players of the 2021 season in terms of average rebounds per game
"""

response = answer(sql_data_path,
                  db_schema,
                  sql_model,
                  sql_token,
                  sql_prompt,
                  llm,
                  llm_prompt,
                  q)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   40312.73 ms
llama_print_timings:      sample time =     213.47 ms /    97 runs   (    2.20 ms per token,   454.39 tokens per second)
llama_print_timings: prompt eval time =   45160.85 ms /   117 tokens (  385.99 ms per token,     2.59 tokens per second)
llama_print_timings:        eval time =   59943.49 ms /    96 runs   (  624.41 ms per token,     1.60 tokens per second)
llama_print_timings:       total time =  106411.32 ms /   213 tokens


The top 5 players of the 2021 season in terms of average rebounds per game are:

1. Clint Capela - 14.0
2. Rudy Gobert - 13.96
3. Jonas Valanciunas - 12.32
4. Domantas Sabonis - 12.02
5. Nikola Jokic - 11.73

These players had the highest average rebounds per game in the 2021 season.


In [64]:
q = """
Show me the 10 best performances in terms of point scored in a single game
"""

response = answer(sql_data_path,
                  db_schema,
                  sql_model,
                  sql_token,
                  sql_prompt,
                  llm,
                  llm_prompt,
                  q)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   40312.73 ms
llama_print_timings:      sample time =     274.74 ms /   139 runs   (    1.98 ms per token,   505.94 tokens per second)
llama_print_timings: prompt eval time =   67183.04 ms /   140 tokens (  479.88 ms per token,     2.08 tokens per second)
llama_print_timings:        eval time =   87367.68 ms /   138 runs   (  633.10 ms per token,     1.58 tokens per second)
llama_print_timings:       total time =  144458.01 ms /   278 tokens


The top 10 performances in terms of points scored in a single game are:

1. Wilt Chamberlain - 100
2. Kobe Bryant - 81
3. Wilt Chamberlain - 78
4. Wilt Chamberlain - 73
5. David Thompson - 73
6. Luka Doncic - 73
7. Wilt Chamberlain - 72
8. Elgin Baylor - 71
9. David Robinson - 71
10. Wilt Chamberlain - 73

Note: There are multiple ties for the top 5 spots, but I've listed all the players who achieved those scores in my response.


In [65]:
q = """
Who is the player with the most assists in a season?
"""

response = answer(sql_data_path,
                  db_schema,
                  sql_model,
                  sql_token,
                  sql_prompt,
                  llm,
                  llm_prompt,
                  q)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   40312.73 ms
llama_print_timings:      sample time =      43.84 ms /    20 runs   (    2.19 ms per token,   456.23 tokens per second)
llama_print_timings: prompt eval time =   24144.97 ms /    64 tokens (  377.27 ms per token,     2.65 tokens per second)
llama_print_timings:        eval time =   11469.43 ms /    19 runs   (  603.65 ms per token,     1.66 tokens per second)
llama_print_timings:       total time =   35889.95 ms /    83 tokens


Steve Nash has the most assists in a season with a total of 771.0 assists.


In [67]:
q = """
Who is the player with the most rebounds in a season?
"""

response = answer(sql_data_path,
                  db_schema,
                  sql_model,
                  sql_token,
                  sql_prompt,
                  llm,
                  llm_prompt,
                  q)

print(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   40312.73 ms
llama_print_timings:      sample time =      54.04 ms /    26 runs   (    2.08 ms per token,   481.16 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =   16070.73 ms /    26 runs   (  618.11 ms per token,     1.62 tokens per second)
llama_print_timings:       total time =   16412.82 ms /    26 tokens


The player with the most rebounds in a season is Shaquille O'Neal with 26.0 rebounds per game.
